In [3]:
from __future__ import annotations

import argparse
import json
import os
import re
import sys
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import pandas as pd

In [20]:
def ts() -> str:
    """Return timestamp string for logging"""
    return f"[{datetime.now().strftime('%Y%m%d %H:%M:%S')}]"

def parse_image_filename(filename: str) -> Optional[Dict[str, str]]:
    """
    Parse image filename like: SM_1_IMG_1234_20170315_143027__000001.jpg
    Returns dict with: location, image_name, date_str, time_str
    """
    # Pattern: SM_X_<imagename>_YYYYMMDD_HHMMSS__NNNNNN.ext
    # Updated to handle location like SM_1, SM_2, etc.
    pattern = r'^(SM_\d+)_(.+?)_(\d{8})_(\d{6})__\d{6}\.\w+$'
    match = re.match(pattern, filename)
    if match:
        return {
            'location': match.group(1),      # e.g., SM_1
            'image_name': match.group(2),    # e.g., IMG_1234
            'date_str': match.group(3),      # e.g., 20170315 (YYYYMMDD)
            'time_str': match.group(4)       # e.g., 143027 (HHMMSS)
        }
    return None

def parse_excel_date(date_val) -> Optional[str]:
    """
    Convert Excel date (dd-mm-yyyy or datetime object) to YYYYMMDD string
    Handles multiple input formats from Excel
    """
    if pd.isna(date_val):
        return None
    
    # If it's already a datetime object (common in pandas)
    if isinstance(date_val, datetime):
        return date_val.strftime('%Y%m%d')
    
    # If it's a string in dd-mm-yyyy or dd/mm/yyyy format
    if isinstance(date_val, str):
        date_str = date_val.strip()
        # Try dd-mm-yyyy
        for sep in ['-', '/']:
            try:
                dt = datetime.strptime(date_str, f'%d{sep}%m{sep}%Y')
                return dt.strftime('%Y%m%d')
            except:
                pass
        # Try yyyy-mm-dd (alternative format)
        try:
            dt = datetime.strptime(date_str, '%Y-%m-%d')
            return dt.strftime('%Y%m%d')
        except:
            pass
    
    return None

def parse_excel_time(time_val) -> Optional[str]:
    """
    Convert Excel time (HH:MM:SS or datetime.time object) to HHMM string
    Only extracts hours and minutes for matching
    """
    if pd.isna(time_val):
        return None
    
    # If it's a datetime.time object (most common from Excel)
    if hasattr(time_val, 'hour') and hasattr(time_val, 'minute'):
        return f"{time_val.hour:02d}{time_val.minute:02d}"
    
    # If it's a datetime object
    if isinstance(time_val, datetime):
        return time_val.strftime('%H%M')
    
    # If it's a pandas Timedelta object
    if isinstance(time_val, pd.Timedelta):
        total_seconds = int(time_val.total_seconds())
        hours = (total_seconds // 3600) % 24
        minutes = (total_seconds % 3600) // 60
        return f"{hours:02d}{minutes:02d}"
    
    # If it's a string in HH:MM:SS format
    if isinstance(time_val, str):
        try:
            parts = time_val.strip().split(':')
            if len(parts) >= 2:
                hh = int(parts[0])
                mm = int(parts[1])
                return f"{hh:02d}{mm:02d}"
        except:
            pass
    
    return None

def time_diff_minutes(time1_hhmm: str, time2_hhmm: str) -> int:
    """
    Calculate difference in minutes between two times in HHMM format
    Returns time1 - time2 in minutes
    """
    try:
        h1, m1 = int(time1_hhmm[:2]), int(time1_hhmm[2:4])
        h2, m2 = int(time2_hhmm[:2]), int(time2_hhmm[2:4])
        
        total_min1 = h1 * 60 + m1
        total_min2 = h2 * 60 + m2
        
        return total_min1 - total_min2
    except:
        return 999  # Return large number if parsing fails

def discover_images(img_dir: str, allowed_exts: set[str]) -> List[Tuple[str, Dict]]:
    """
    Recursively find all images and parse their metadata
    Returns list of (full_path, parsed_metadata)
    """
    images = []
    img_dir_abs = os.path.abspath(img_dir)
    
    print(f"{ts()} Scanning directory: {img_dir_abs}")
    
    for root, dirs, files in os.walk(img_dir_abs):
        for fn in files:
            ext = os.path.splitext(fn)[1].lower()
            if ext in allowed_exts:
                parsed = parse_image_filename(fn)
                if parsed:
                    full_path = os.path.join(root, fn)
                    images.append((full_path, parsed))
                else:
                    # Image doesn't match expected naming pattern
                    pass
    
    return images

def match_label_to_image(
    label_row: pd.Series,
    images: List[Tuple[str, Dict]],
    verbose: bool = False
) -> Optional[str]:
    """
    Find matching image for a label row based on:
    1. Location (exact match) - e.g., SM_1
    2. ImageName (exact match) - e.g., IMG_1234
    3. TrueDate (exact match) - converted to YYYYMMDD
    4. TrueTime (within ±2 minutes) - only HH:MM compared
    
    Returns image path if found, else None
    """
    # Parse label data
    label_location = str(label_row.get('Location', '')).strip()
    label_image_name = str(label_row.get('ImageName', '')).strip()
    label_date = parse_excel_date(label_row.get('TrueDate'))
    label_time = parse_excel_time(label_row.get('TrueTime'))
    
    # Check if all required fields are present
    if not label_location or label_location == 'nan':
        if verbose:
            print(f"{ts()}   Missing Location")
        return None
    if not label_image_name or label_image_name == 'nan':
        if verbose:
            print(f"{ts()}   Missing ImageName")
        return None
    if not label_date:
        if verbose:
            print(f"{ts()}   Missing/Invalid TrueDate")
        return None
    if not label_time:
        if verbose:
            print(f"{ts()}   Missing/Invalid TrueTime")
        return None
    
    if verbose:
        print(f"{ts()}   Looking for: {label_location} / {label_image_name} / {label_date} / {label_time}")
    
    # Search through images
    for img_path, img_meta in images:
        # 1. Location exact match
        if img_meta['location'] != label_location:
            continue
        
        # 2. ImageName exact match
        if img_meta['image_name'] != label_image_name:
            continue
        
        # 3. Date exact match
        if img_meta['date_str'] != label_date:
            continue
        
        # 4. Time within ±2 minutes (only HH and MM)
        img_time_hhmm = img_meta['time_str'][:4]  # Extract HHMM from HHMMSS
        label_time_hhmm = label_time
        
        diff = time_diff_minutes(img_time_hhmm, label_time_hhmm)
        
        if verbose:
            print(f"{ts()}   Checking {os.path.basename(img_path)}: time diff = {diff} minutes")
        
        if -2 <= diff <= 2:
            if verbose:
                print(f"{ts()}   MATCH FOUND: {img_path}")
            return img_path
    
    if verbose:
        print(f"{ts()}   No match found")
    
    return None

In [23]:
def main():
    parser = argparse.ArgumentParser(
        description="Match camera trap labels with processed images and create JSON + updated Excel"
    )
    parser.add_argument("--label-file", required=True, help="Path to Excel label file")
    parser.add_argument("--img-dir", required=True, help="Root directory containing processed images")
    parser.add_argument("--output-json", required=True, help="Output JSON file path")
    parser.add_argument("--output-excel", required=True, help="Output Excel file with image paths")
    parser.add_argument("--exts", default="jpg,jpeg", help="Comma-separated image extensions (default: jpg,jpeg)")
    parser.add_argument("--verbose", action="store_true", help="Enable verbose output for debugging")
    parser.add_argument("--sample", type=int, default=0, help="Process only first N rows (for testing)")
    args = parser.parse_args()
    
    if not PANDAS_OK:
        print(f"{ts()} ERROR: pandas required but not available")
        print(f"{ts()} Install with: pip install pandas openpyxl")
        return
    
    # Normalize extensions
    exts = {("." + e.strip().lower().lstrip(".")) for e in args.exts.split(",") if e.strip()}
    
    print(f"{ts()} ========================================")
    print(f"{ts()} Camera Trap Label-Image Matcher")
    print(f"{ts()} ========================================")
    print(f"{ts()} Label file: {args.label_file}")
    print(f"{ts()} Image directory: {args.img_dir}")
    print(f"{ts()} Output JSON: {args.output_json}")
    print(f"{ts()} Output Excel: {args.output_excel}")
    print(f"{ts()} Extensions: {sorted(exts)}")
    print(f"{ts()} ========================================")
    
    # Load Excel file
    print(f"{ts()} Loading label file...")
    try:
        df = pd.read_excel(args.label_file)
    except Exception as e:
        print(f"{ts()} ERROR loading Excel: {e}")
        return
    
    print(f"{ts()} Loaded {len(df)} label rows")
    print(f"{ts()} Columns found: {list(df.columns)}")
    
    # Verify required columns exist
    required_cols = ['Location', 'ImageName', 'TrueDate', 'TrueTime']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"{ts()} ERROR: Missing required columns: {missing_cols}")
        return
    
    # Optional columns for output
    optional_cols = ['CommonName', 'ScientificName', 'Number', 'Sex', 'Notes', 'BestPhoto']
    
    # Limit to sample if specified
    if args.sample > 0:
        print(f"{ts()} WARNING: Processing only first {args.sample} rows (--sample mode)")
        df = df.head(args.sample)
    
    # Discover all images
    print(f"{ts()} Scanning images in: {args.img_dir}")
    images = discover_images(args.img_dir, exts)
    print(f"{ts()} Found {len(images)} images with valid filenames")
    
    if len(images) == 0:
        print(f"{ts()} WARNING: No images found. Check the image directory and file naming pattern.")
        return
    
    # Show sample images
    print(f"{ts()} Sample images (first 5):")
    for i, (path, meta) in enumerate(images[:5]):
        print(f"{ts()}   {i+1}. {os.path.basename(path)}")
        print(f"{ts()}      Location: {meta['location']}, Name: {meta['image_name']}, Date: {meta['date_str']}, Time: {meta['time_str']}")
    
    # Matching process
    print(f"{ts()} ========================================")
    print(f"{ts()} Starting matching process...")
    print(f"{ts()} ========================================")
    
    matches = []
    matched_count = 0
    
    # Add ImagePath column to dataframe
    df['ImagePath'] = None
    
    for idx, row in df.iterrows():
        if args.verbose or (idx + 1) % 100 == 1:
            print(f"{ts()} Processing row {idx + 1}/{len(df)}...")
        
        img_path = match_label_to_image(row, images, verbose=args.verbose)
        
        if img_path:
            matched_count += 1
            df.at[idx, 'ImagePath'] = img_path
            
            # Build JSON entry with available fields
            json_entry = {
                'image_path': img_path,
            }
            
            # Add optional fields if they exist in the dataframe
            for col in optional_cols:
                if col in df.columns:
                    val = row.get(col, '')
                    # Convert to string, handle NaN
                    if pd.isna(val):
                        json_entry[col] = ''
                    else:
                        json_entry[col] = str(val)
                else:
                    json_entry[col] = ''
            
            matches.append(json_entry)
        
        # Progress updates every 100 rows
        if (idx + 1) % 100 == 0:
            print(f"{ts()} Progress: {idx + 1}/{len(df)} labels processed, {matched_count} matched ({matched_count/(idx+1)*100:.1f}%)")
    
    print(f"{ts()} ========================================")
    print(f"{ts()} Matching complete!")
    print(f"{ts()} ========================================")
    
    # Write JSON output
    print(f"{ts()} Writing JSON to: {args.output_json}")
    try:
        os.makedirs(os.path.dirname(args.output_json), exist_ok=True)
        with open(args.output_json, 'w') as f:
            json.dump(matches, f, indent=2)
        print(f"{ts()} JSON written successfully ({len(matches)} entries)")
    except Exception as e:
        print(f"{ts()} ERROR writing JSON: {e}")
    
    # Write updated Excel
    print(f"{ts()} Writing Excel to: {args.output_excel}")
    try:
        os.makedirs(os.path.dirname(args.output_excel), exist_ok=True)
        df.to_excel(args.output_excel, index=False)
        print(f"{ts()} Excel written successfully")
    except Exception as e:
        print(f"{ts()} ERROR writing Excel: {e}")
    
    # Final summary
    print(f"{ts()} ========================================")
    print(f"{ts()} FINAL SUMMARY")
    print(f"{ts()} ========================================")
    print(f"{ts()} Total labels processed: {len(df)}")
    print(f"{ts()} Successfully matched: {matched_count}")
    print(f"{ts()} Unmatched: {len(df) - matched_count}")
    print(f"{ts()} Match rate: {matched_count/len(df)*100:.1f}%")
    print(f"{ts()} ========================================")
    print(f"{ts()} DONE!")

In [26]:
# You can comment out this section when running from command line
sys.argv = [
    "Convert_label_csv_file_to_json_format_v5.ipynb",
    "--label-file", "/home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Final_Label.xlsx",
    "--img-dir", "/home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Processed_Images/",
    "--output-json", "/home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Label.json",
    "--output-excel", "/home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Final_Label_with_paths.xlsx",
    # "--verbose",  # Uncomment for detailed debugging
    # "--sample", "10",  # Uncomment to test with first 10 rows only
]
main()

[20251102 06:43:54] ========================================
[20251102 06:43:54] Camera Trap Label-Image Matcher
[20251102 06:43:54] ========================================
[20251102 06:43:54] Label file: /home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Final_Label.xlsx
[20251102 06:43:54] Image directory: /home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Processed_Images/
[20251102 06:43:54] Output JSON: /home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Label.json
[20251102 06:43:54] Output Excel: /home/hice1/ssinha348/scratch/stonemt_cameratrap/Camera Trap Photos/Final_Label_with_paths.xlsx
[20251102 06:43:54] Extensions: ['.jpeg', '.jpg']
[20251102 06:43:54] ========================================
[20251102 06:43:54] Loading label file...
[20251102 06:43:57] Loaded 23351 label rows
[20251102 06:43:57] Columns found: ['Location', 'ImageName', 'CameraDate', 'CameraDate_InDateFormat', 'CameraTime', 'TrueDate', 'TrueTime', 'Common